In [10]:
import sys

import matplotlib.pyplot as plt
import cebra.data
import cebra.datasets
import cebra.integrations
from cebra import CEBRA
import matplotlib.pyplot as plt
import pickle
from percephone.analysis.utils import get_zscore
from percephone.cebra.CEBRA_trials_viz import get_recs_dict
%matplotlib notebook

In [2]:
recs = get_recs_dict("Célien")

20220710_4445_00_synchro20220715_4456_00_synchro
20220930_4745_01_synchro

20220930_4756_01_synchro
20221004_4754_01_synchro
20221205_4939_04_synchro
20231008_5890_03_synchro
20231009_5886_00_synchro
20231009_5896_04_synchro
20231025_5893_00_synchro
20231031_5879_00_synchro
20231102_5889_03_synchro
Behavioural information already incorporated in the analog.
Behavioural information already incorporated in the analog.
Behavioural information already incorporated in the analog.
Behavioural information already incorporated in the analog.
Behavioural information already incorporated in the analog.
Behavioural information already incorporated in the analog.
Behavioural information already incorporated in the analog.
Behavioural information already incorporated in the analog.
Behavioural information already incorporated in the analog.
Behavioural information already incorporated in the analog.
Behavioural information already incorporated in the analog.
20231104_5873_04_synchro
Behavioural inf

In [11]:
hypo_only = False

names_wt, datas_wt, labels_wt = [], [], []
names_ko, datas_ko, labels_ko = [], [], []

for rec in recs.values():
    if not (rec.genotype == "KO" and hypo_only):
        name = rec.filename
        label = rec.detected_stim.astype(int)
        data = get_zscore(rec, exc_neurons=True, inh_neurons=True, time_span="stim", window=0.5, estimator="Mean", sort=False, amp_sort=False, single_frame_estimator=True)[0].T
        
        if rec.genotype == "WT":
            names_wt.append(name)
            datas_wt.append(data)
            labels_wt.append(label)
        else:
            names_ko.append(name)
            datas_ko.append(data)
            labels_ko.append(label)
    else:
        continue

print(f"WT: {names_wt}")
print(f"KO: {names_ko}")

WT: [4456, 4745, 4939, 5886, 5896, 5879, 5889, 5873, 6606]
KO: [4445, 4756, 4754, 5890, 5893, 5881, 6601, 6609, 6611]


## Single-animal model

In [12]:
max_iter=1000

embeddings_wt = dict()
embeddings_ko = dict()

# Single session training
for name, X, y in zip(names_wt, datas_wt, labels_wt):
    # Fit one CEBRA model per session (i.e., per rat)
    print(f"Fitting CEBRA for {name}")
    cebra_model = CEBRA(model_architecture='offset10-model',
                        batch_size=512,
                        learning_rate=3e-4,
                        temperature=1,
                        output_dimension=3,
                        max_iterations=max_iter,
                        distance='cosine',
                        conditional='time_delta',
                        device='cuda_if_available',
                        verbose=True,
                        time_offsets=10)

    cebra_model.fit(X, y)
    embeddings_wt[name] = cebra_model.transform(X)
for name, X, y in zip(names_ko, datas_ko, labels_ko):
    # Fit one CEBRA model per session (i.e., per rat)
    print(f"Fitting CEBRA for {name}")
    cebra_model = CEBRA(model_architecture='offset10-model',
                        batch_size=512,
                        learning_rate=3e-4,
                        temperature=1,
                        output_dimension=3,
                        max_iterations=max_iter,
                        distance='cosine',
                        conditional='time_delta',
                        device='cuda_if_available',
                        verbose=True,
                        time_offsets=10)

    cebra_model.fit(X, y)
    embeddings_ko[name] = cebra_model.transform(X)


# Align the single session embeddings to the first rat
alignment = cebra.data.helper.OrthogonalProcrustesAlignment()
first_rat_wt = list(embeddings_wt.keys())[0]
first_rat_ko = list(embeddings_ko.keys())[0]

for j, rat_name in enumerate(list(embeddings_wt.keys())[1:]):
    embeddings_wt[f"{rat_name}"] = alignment.fit_transform(
        embeddings_wt[first_rat_wt], embeddings_wt[rat_name], labels_wt[0], labels_wt[j+1])
for j, rat_name in enumerate(list(embeddings_ko.keys())[1:]):
    embeddings_ko[f"{rat_name}"] = alignment.fit_transform(
        embeddings_ko[first_rat_ko], embeddings_ko[rat_name], labels_ko[0], labels_ko[j+1])

# Save embeddings in current folder
# with open('embeddings_wt.pkl', 'wb') as f:
#     pickle.dump(embeddings_wt, f)
# with open('embeddings_ko.pkl', 'wb') as f:
#     pickle.dump(embeddings_ko, f)


Fitting CEBRA for 4456


pos: -0.7998 neg:  6.6749 total:  5.8751 temperature:  1.0000: 100%|██████████| 1000/1000 [00:17<00:00, 58.61it/s]


Fitting CEBRA for 4745


pos: -0.7580 neg:  6.6270 total:  5.8690 temperature:  1.0000: 100%|██████████| 1000/1000 [00:16<00:00, 61.36it/s]


Fitting CEBRA for 4939


...

pos: -0.8599 neg:  6.6439 total:  5.7840 temperature:  1.0000: 100%|██████████| 1000/1000 [00:14<00:00, 66.97it/s]


Fitting CEBRA for 6609


pos: -0.8747 neg:  6.9797 total:  6.1050 temperature:  1.0000: 100%|██████████| 1000/1000 [00:15<00:00, 65.76it/s]


Fitting CEBRA for 6611


pos: -0.8759 neg:  6.9388 total:  6.0629 temperature:  1.0000: 100%|██████████| 1000/1000 [00:15<00:00, 66.35it/s]


## Multi-animal models

In [13]:
multi_embeddings_wt = dict()
multi_embeddings_ko = dict()

# Multisession training
multi_cebra_model_wt = CEBRA(model_architecture='offset10-model',
                    batch_size=512,
                    learning_rate=3e-4,
                    temperature=1,
                    output_dimension=3,
                    max_iterations=max_iter,
                    distance='cosine',
                    conditional='time_delta',
                    device='cuda_if_available',
                    verbose=True,
                    time_offsets=10)
multi_cebra_model_ko = CEBRA(model_architecture='offset10-model',
                    batch_size=512,
                    learning_rate=3e-4,
                    temperature=1,
                    output_dimension=3,
                    max_iterations=max_iter,
                    distance='cosine',
                    conditional='time_delta',
                    device='cuda_if_available',
                    verbose=True,
                    time_offsets=10)

# Provide a list of data, i.e. datas = [data_a, data_b, ...]
multi_cebra_model_wt.fit(datas_wt, labels_wt)
multi_cebra_model_ko.fit(datas_wt, labels_wt)

# Transform each session with the right model, by providing the corresponding session ID
for i, (name, X) in enumerate(zip(names_wt, datas_wt)):
    multi_embeddings_wt[name] = multi_cebra_model_wt.transform(X, session_id=i)
for i, (name, X) in enumerate(zip(names_ko, datas_ko)):
    multi_embeddings_ko[name] = multi_cebra_model_ko.transform(X, session_id=i)

# Save embeddings in current folder
with open('multi_embeddings_wt.pkl', 'wb') as f:
    pickle.dump(multi_embeddings_wt, f)
with open('multi_embeddings_ko.pkl', 'wb') as f:
    pickle.dump(multi_embeddings_ko, f)

NotImplementedError: Multisession implementation does not support discrete index yet.